# Create Model Area

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-03-19

In [ ]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Create Model Area
# Author: Timm Nawrocki
# Created on: 2019-03-19
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Create Model Area" generates a polygon from an array of x, y coordinates that will serve as the general modeling area for data download and processing.
# ---------------------------------------------------------------------------

## 1. Initialize Environment

In [1]:
# Import packages
import arcpy
import fileinput
import numpy as np
import os

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set arcpy working environment
arcpy.env.workspace = "K:/VegetationEcology/BristolBay_Vegetation/Project_GIS/BristolBay_Vegetation.gdb"

# Define input array
point_array = [[-160.00218, 61.52927],
               [-152.79699, 61.07407],
               [-152.70467, 58.66074],
               [-157.69984, 56.33955],
               [-159.49354, 56.97233],
               [-163.09044, 58.59629],
               [-160.84181, 61.57616]]

# Define input datasets
alaska_boundary = 'K:/VegetationEcology/BristolBay_Vegetation/Project_GIS/Data_Input/source_data/boundaries/Alaska_63360_ExcludingSmallIslands.shp'

# Define spatial references
geographic_reference = 4326 #GCS WGS 1984
projected_reference = 3338 #NAD 1983 Alaska Albers

# Define output feature class
model_area_full = "SouthwestAlaska_ModelArea_Full"
model_area_clipped = "SouthwestAlaska_ModelArea_Clipped"

In [2]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Create Polygon

In [3]:
# Define function to create polygon feature class
def create_polygon(**kwargs):
    """
    Description: creates a polygon feature class with vertices read from an array
    Inputs: 'point_array' -- an array of x, y coordinates
            'output_array' -- the output should be an array with a full feature class and a clipped feature class
            'output_path' -- the output path is the directory in which the new feature class is created
    """
    # Create a new polygon feature class
    arcpy.CreateFeatureclass_management(kwargs['output_path'],
                                        'modelArea_initial',
                                        'POLYGON')
    # Define spatial reference and add projection to new feature class
    spatial_reference = arcpy.SpatialReference(kwargs['spatial_reference'])
    output_file = os.path.join(kwargs['output_path'], kwargs['output'])
    arcpy.DefineProjection_management(output_file, spatial_reference)
    # Initiate an insert cursor and insert an array of vertices for a polygon
    with arcpy.da.InsertCursor(output_file, ["SHAPE@"]) as cursor:
        array = arcpy.Array()
        for point in kwargs['point_array']:
            X = point[0]
            Y = point[1]
            array.add(arcpy.Point(X, Y))
        cursor.insertRow([arcpy.Polygon(array)])
    # Project polygon
    output_feature = os.path.join(kwargs['output_path'], os.path.split(kwargs['output'])[1]
    
    # Delete intermediate dataset
    out_process = arcpy.Delete_management(os.path.join(kwargs['output_path'], 'modelArea_initial'))
    # Return success message to the user
    print("------------")
    out_process = "Successfully created polygon feature"
    return out_process

In [4]:
# Create key word arguments
create_polygon_kwargs = {'point_array' : point_array,
                         'output' : model_area,
                         'output_path' : arcpy.env.workspace,
                         'spatial_reference' : geographic_reference}

# Process the create polygon function with the point array
arcpy_geoprocessing(create_polygon, check_input = False, **create_polygon_kwargs)

------------
Successfully created polygon feature


## 3. Process Polygon